### Summary

- Detect face using pretrained model

### Face Detection

Import asset dump

In [4]:
import os
import cv2
import sys
from zipfile import ZipFile
from urllib.request import urlretrieve

def download_and_unzip(url, save_path):
    print(f"Downloading and extracting assests....", end="")

    # Downloading zip file using urllib package.
    urlretrieve(url, save_path)

    try:
        # Extracting zip file using the zipfile package.
        with ZipFile(save_path) as z:
            # Extract ZIP file contents in the same directory.
            z.extractall(os.path.split(save_path)[0])

        print("Done")

    except Exception as e:
        print("\nInvalid file.", e)


URL = r"https://www.dropbox.com/s/efitgt363ada95a/opencv_bootcamp_assets_12.zip?dl=1"

asset_zip_path = os.path.join(os.getcwd(), f"opencv_bootcamp_assets_12.zip")

# Download if assest ZIP does not exists.
if not os.path.exists(asset_zip_path):
    download_and_unzip(URL, asset_zip_path)

Define camera and its camera window:

In [36]:
cap = cv2.VideoCapture(0) # default camera

win_name = "Camera"

Define the model being used

In [28]:
# use caffe model
model = cv2.dnn.readNetFromCaffe("deploy.prototxt", "res10_300x300_ssd_iter_140000_fp16.caffemodel")

Define hyperparameters of the model

In [29]:
width = 300 # width of input training image
height = 300 # height of input training image
mean = [104, 117, 123] # rgb colors of all training images on average
conf_threshold = 0.6 # confidence threshold

Loop camera with model:

In [37]:
cv2.namedWindow(win_name, cv2.WINDOW_NORMAL)

while cap.isOpened():

    # read each frame of video
    retval, frame = cap.read()

    if not retval:
        break
        
    # start time for fps
    timer = cv2.getTickCount()

    # get dimensions of frame
    frame_height = frame.shape[0]
    frame_width = frame.shape[1]

    # create a blob (pixels with similar characteristics) in the frame
    blob = cv2.dnn.blobFromImage(frame, 1, (width, height), mean, swapRB = False, crop = False)

    # run the defined model
    model.setInput(blob)
    detections = model.forward()

    # loop for all detections, see the confidence in each pixel frame
    for i in range(detections.shape[2]):
        conf = detections[0, 0, i, 2]
        if conf > conf_threshold:

            # find pixel corner coordinates
            x1 = int(detections[0, 0, i, 3] * frame_width)
            y1 = int(detections[0, 0, i, 4] * frame_height)
            x2 = int(detections[0, 0, i, 5] * frame_width)
            y2 = int(detections[0, 0, i, 6] * frame_height)

            # make bounding box based on corner coordinates
            cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0))
            label = "face %.4f" % conf
            label_size, base_line = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
            cv2.rectangle(frame, (x1, y1 - label_size[1]), (x1 + label_size[0], y1 + base_line), (255, 0, 0), cv2.FILLED)
            cv2.putText(frame, label, (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0))

    # report inference time
    inference_time, fill = model.getPerfProfile()
    inf_label = "Inference time; %.3f ms" % (inference_time * 1000 / cv2.getTickFrequency())
    cv2.putText(frame, inf_label, (20, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255))

    # get and display fps
    fps = cv2.getTickFrequency() / (cv2.getTickCount() - timer)
    cv2.putText(frame, "FPS: %.2f" % fps, (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255))

    if cv2.waitKey(1) == "q" or cv2.waitKey(1) == 27:
        break

    # show frame
    cv2.imshow(win_name, frame)

cap.release()
cv2.destroyAllWindows()